# ASP Bundle Adjust Plotting
## Examples for BlackSky Easton Glacier test case (n=20)
David Shean  
12/24/22

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors
import contextily as ctx

In [ ]:
from asp_plot_util import *

In [ ]:
#topdir = '/Users/dshean/scr/BlackSky/DAN_TUM_multiple_opportunities/Tuolumne'
topdir = '/Users/dshean/scr/BlackSky/DAN_TUM_multiple_opportunities/Dana'

In [ ]:
topdir = '/Users/dshean/scr/BlackSky/EastonGlacier_20220918-20221012/non-ortho'
topdir = '/Users/dshean/scr/BlackSky/EastonGlacier_20220918-20221012/non-ortho_20230102'

In [ ]:
topdir = '/Users/dshean/scr/BlackSky/Utqiagvik_20220425_stereo/BSG-STEREO-102-20220425-215106-22900060-stereo_reorder_20230119/'

In [ ]:
topdir = '/Users/dshean/scr/hma_glof_samples_for_asp_plot'

In [ ]:
#topdir = '/Users/dshean/scr/BlackSky/GM_SnowOff_202208'

In [ ]:
cd $topdir

In [ ]:
#Old filenames - Easton testcase
#ba_prefix = 'ba_all/ba_all'
#ba_prefix = 'ba_all/ba_all_tri_weight'
#ba_prefix = 'ba_all/ba_all_tri_weight_pc_align'
#ba_prefix="ba_all_nadirpinhole/ba_all_nadirpinhole_tri_weight"
#ba_prefix="ba_all_maskref/ba_all_maskref_tri_weight"
#ba_prefix="ba_all_maskref_nadirpinhole/ba_all_maskref_nadirpinhole_tri_weight"
ba_prefix="ba_all_maskref/ba_all_maskref_tri_weight_pc_align"
#ba_prefix="ba_all_maskref_rpc/ba_all_maskref_rpc_tri_weight"

In [ ]:
#New filenames
ba_prefix="ba/ba_all_rpc"
ba_prefix="ba/ba_all_nadirpinhole"

In [ ]:
ba_prefix="ba/ba_nadirpinhole_hfdem"
#ba_brefix="ba/ba_ip2k_nadirpinhole"

In [ ]:
ba_prefix = 'ba/ba_csm'

In [ ]:
map_crs = 'EPSG:32610'
#map_crs = 'EPSG:32604'

In [ ]:
map_crs = 'EPSG:32645'
#map_crs = 'EPSG:32604'

In [ ]:
refdem = 'COP30_lzw-adj_proj.tif'
#Masked version
#refdem = 'COP30_lzw-adj_proj_ref.tif'

In [ ]:
refdem = 'GongbatongshaTsho_COP30_lzw-adj_proj.tif'

In [ ]:
source = ctx.providers.Esri.WorldImagery
#source = ctx.providers.Stamen.Terrain

In [ ]:
ctx_kwargs = {'crs':map_crs, 'source':source, 'attribution_size':0, 'alpha':0.5}

In [ ]:
#Use to compare multi-stage bundle_adjust results
two_stage = False

## Residuals

In [ ]:
def read_residuals(csv_fn):
    resid_cols=['lon', 'lat', 'height_above_datum', 'mean_residual', 'num_observations']
    resid_df = pd.read_csv(csv_fn, skiprows=2, names=resid_cols)
    #Need the astype('str') to handle cases where column has dtype of int (without the # from DEM appended to some rows)
    resid_df['from_DEM'] = resid_df['num_observations'].astype('str').str.contains('# from DEM')
    resid_df['num_observations'] = resid_df['num_observations'].astype('str').str.split('#', expand=True)[0].astype(int)
    resid_gdf = gpd.GeoDataFrame(resid_df, geometry=gpd.points_from_xy(resid_df['lon'], resid_df['lat'], crs='EPSG:4326'))
    return resid_gdf

In [ ]:
resid_init_csv = ba_prefix+'-initial_residuals_pointmap.csv'
resid_final_csv = ba_prefix+'-final_residuals_pointmap.csv'

In [ ]:
#This compares the initial bundle_adjust output with post-pc_align bundle_adjust output
if two_stage:
    resid_init_csv = ba_prefix+'-final_residuals_pointmap.csv'
    resid_final_csv = ba_prefix+'_pc_align-final_residuals_pointmap.csv'

In [ ]:
resid_init = read_residuals(resid_init_csv)
resid_final = read_residuals(resid_final_csv)

In [ ]:
#Computer center for map plots later
centroid_gdf = resid_final.to_crs(map_crs).dissolve().centroid

In [ ]:
resid_init.describe()

In [ ]:
resid_final.describe()

In [ ]:
def resid_plot(init, final, col='mean_residual', clip_final=True, lognorm=False, clim=None, cmap='inferno'):
    f, axa = plt.subplots(1,2, figsize=(10,3), sharex=True, sharey=True)
    if clim is None:
        clim_init = get_clim(init[col], perc=(0,98))
        clim_final = get_clim(final[col], perc=(0,98))
        vmin = min(clim_init[0], clim_final[0])
        vmax = max(clim_init[1], clim_final[1])
    else:
        vmin, vmax = clim
    print(vmin, vmax)
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    if lognorm:
        norm = matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax)
    plot_kw = {'cmap':cmap, 'norm':norm, 's':1, 'legend':True, 'legend_kwds':{'label': col}}
    final.sort_values(by=col).to_crs(map_crs).plot(ax=axa[1], column=col, **plot_kw)
    ctx.add_basemap(ax=axa[1], **ctx_kwargs)
    if clip_final:
        axa[0].autoscale(False)
    init.sort_values(by=col).to_crs(map_crs).plot(ax=axa[0], column=col, **plot_kw)
    ctx.add_basemap(ax=axa[0], **ctx_kwargs)
    axa[0].set_title(f'Initial Residuals (n={init.shape[0]})')
    axa[1].set_title(f'Final Residuals (n={final.shape[0]})')
    plt.tight_layout()

In [ ]:
resid_plot(resid_init, resid_final, col='mean_residual', lognorm=False)

In [ ]:
resid_plot(resid_init, resid_final, col='mean_residual', lognorm=True)

In [ ]:
resid_plot(resid_init, resid_final, col='num_observations')

## Isolate points used during `--heights-from-DEM`
Most relelvant when refDEM was masked over changing surfaces

In [ ]:
if 'from_DEM' in resid_init.columns:
    idx1 = resid_init['from_DEM']
    idx2 = resid_final['from_DEM']

In [ ]:
if idx1.any() and idx2.any():
    resid_plot(resid_init[idx1], resid_final[idx2], col='mean_residual', lognorm=True)

In [ ]:
if ~idx1.any() and ~idx2.any():
    resid_plot(resid_init[~idx1], resid_final[~idx2], col='mean_residual', lognorm=True)

## geodiff output

In [ ]:
def read_geodiff(csv_fn):
    resid_cols=['lon', 'lat', 'diff']
    resid_df = pd.read_csv(csv_fn, comment='#', names=resid_cols)
    resid_gdf = gpd.GeoDataFrame(resid_df, geometry=gpd.points_from_xy(resid_df['lon'], resid_df['lat'], crs='EPSG:4326'))
    return resid_gdf

In [ ]:
#geodiff_csv = ba_prefix+f'-final_residuals_pointmap__{os.path.splitext(refdem)[0]}-diff.csv'
geodiff_init_csv = ba_prefix+'-initial_residuals_pointmap-diff.csv'
geodiff_final_csv = ba_prefix+'-final_residuals_pointmap-diff.csv'

In [ ]:
#This compares the initial bundle_adjust output with post-pc_align bundle_adjust output
if two_stage:
    geodiff_init_csv = ba_prefix+'-final_residuals_pointmap-diff.csv'
    resid_final_csv = ba_prefix+'_pc_align-final_residuals_pointmap-diff.csv'

In [ ]:
if os.path.exists(geodiff_init_csv) and os.path.exists(geodiff_final_csv):
    geodiff_init = read_geodiff(geodiff_init_csv)
    geodiff_final = read_geodiff(geodiff_final_csv)
    geodiff_init.describe()   
    geodiff_final.describe()
    resid_plot(geodiff_init, geodiff_final, col='diff', clim=(-15, 15), cmap='RdYlBu')

## Mapproject Residuals

In [ ]:
def read_mapproj_match_offset(csv_fn):
    resid_cols=['lon', 'lat', 'height_above_datum', 'mapproj_ip_dist_meters']
    resid_df = pd.read_csv(csv_fn, skiprows=2, names=resid_cols)
    resid_gdf = gpd.GeoDataFrame(resid_df, geometry=gpd.points_from_xy(resid_df['lon'], resid_df['lat'], crs='EPSG:4326'))
    return resid_gdf

In [ ]:
mapproj_match_offset_txt = ba_prefix+'-mapproj_match_offsets.txt'

In [ ]:
if os.path.exists(mapproj_match_offset_txt):
    mapproj_match_offset = read_mapproj_match_offset(mapproj_match_offset_txt)
    mapproj_match_offset.describe()
    col='mapproj_ip_dist_meters'


In [ ]:
mapproj_match_offset.sort_values(by=col, ascending=True).to_crs(map_crs).plot(column=col, legend=True)
mapproj_match_offset.sort_values(by=col, ascending=False).to_crs(map_crs).plot(column=col, legend=True)

In [ ]:
mapproj_match_offset.sort_values(by=col, ascending=True).to_crs(map_crs).plot(column=col, norm=matplotlib.colors.LogNorm(), legend=True)
mapproj_match_offset.sort_values(by=col, ascending=False).to_crs(map_crs).plot(column=col, norm=matplotlib.colors.LogNorm(), legend=True)

In [ ]:
resid_plot(mapproj_match_offset, mapproj_match_offset, col='mapproj_ip_dist_meters', lognorm=False)

In [ ]:
resid_plot(mapproj_match_offset, mapproj_match_offset, col='mapproj_ip_dist_meters', lognorm=True)

## Plot camera positions

In [ ]:
def read_cameras(csv_fn):
    cam_cols=['input_cam_file','x','y','z','r11','r12','r13','r21','r22','r23','r31','r32','r33']
    cam_df = pd.read_csv(csv_fn, header=0, names=cam_cols, index_col='input_cam_file')
    global_id = cam_df.index.to_series().str.split('BSG', expand=True)[1].str.split('-', expand=True)[1].astype('int') - 100
    cam_df['global_id'] = global_id
    cam_gdf = gpd.GeoDataFrame(cam_df, geometry=gpd.points_from_xy(cam_df['x'], cam_df['y'], cam_df['z'], crs='EPSG:4978'))
    return cam_gdf

In [ ]:
cam_init_csv = ba_prefix+'-initial-cameras.csv'
cam_final_csv = ba_prefix+'-final-cameras.csv'

In [ ]:
#This compares the initial bundle_adjust output with post-pc_align bundle_adjust output
if two_stage:
    cam_init_csv = ba_prefix+'-final-cameras.csv'
    cam_final_csv = ba_prefix+'_pc_align-final-cameras.csv'

In [ ]:
cam_init_gdf = read_cameras(cam_init_csv)
cam_final_gdf = read_cameras(cam_final_csv)

In [ ]:
cam_delta = cam_init_gdf[['x','y','z']] - cam_final_gdf[['x','y','z']]
#The .values here drops the indices (needed when tsai filenames are different at different stages)
if two_stage:
    cam_delta = cam_init_gdf[['x','y','z']].values - cam_final_gdf[['x','y','z']].values
cam_final_gdf['diff_m'] = np.sqrt(np.square(cam_delta).sum(axis=1))

In [ ]:
#For some reason, this doesn't yield same results as above
cam_final_gdf['diff_m_2'] = cam_final_gdf.distance(cam_init_gdf) #align=True

In [ ]:
#cam_init_idx = cam_init['input_cam_file'].str.split('/', expand=True)

### Determine relative local time and time offsets

In [ ]:
temp = cam_final_gdf.index.to_series().str.split('BSG', expand=True)[1].str.split('-', expand=True).loc[:,2:3]
cam_final_gdf['dt'] = pd.to_datetime(temp[2] + temp[3], utc=True)

In [ ]:
min_dt_str = cam_final_gdf['dt'].min().strftime('%Y-%m-%d %H:%M')

In [ ]:
cam_final_gdf['dt_local'] = cam_final_gdf['dt'].dt.tz_convert('America/Denver')

In [ ]:
cam_final_gdf['dt_diff'] = cam_final_gdf['dt'] - cam_final_gdf['dt'].min()

In [ ]:
cam_final_gdf['time'] = cam_final_gdf['dt'].dt.time

In [ ]:
cam_final_gdf['hr'] = ((cam_final_gdf['dt'] - cam_final_gdf['dt'].dt.normalize()) / pd.Timedelta('1 second')).astype(int) / 3600

In [ ]:
cam_final_gdf['hr_local'] = ((cam_final_gdf['dt_local'] - cam_final_gdf['dt_local'].dt.normalize()) / pd.Timedelta('1 second')).astype(int) / 3600

In [ ]:
#cam_final_gdf['time_diff'] = cam_final_gdf['time_diff'] - cam_final_gdf['time_diff'].min()

In [ ]:
cam_final_gdf['dt_diff_days'] = cam_final_gdf['dt_diff'].dt.total_seconds()/86400

In [ ]:
cam_final_gdf['mx'] = cam_final_gdf.to_crs(map_crs).geometry.x.values
cam_final_gdf['my'] = cam_final_gdf.to_crs(map_crs).geometry.y.values

In [ ]:
cam_final_gdf.describe()

In [ ]:
ax = cam_final_gdf.plot.scatter(x='mx', y='my', c='hr_local', s=36, cmap='twilight', edgecolor='k', vmin=0, vmax=24)
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)
ax.set_title('Local time of acquisition')

In [ ]:
ax = cam_final_gdf.plot.scatter(x='mx', y='my', c='dt_diff_days', s=36, cmap='inferno', edgecolor='k')
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)
ax.set_title('Time offset relative to '+min_dt_str)

In [ ]:
ax = cam_final_gdf.plot.scatter(x='mx', y='my', c='dt_diff', s=36, cmap='inferno', edgecolor='k')
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)

In [ ]:
ax = cam_final_gdf.plot.scatter(x='mx', y='my', c='dt', s=36, cmap='inferno', edgecolor='k')
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(c=cam_final_gdf['dt'], cmap='inferno', legend='True', edgecolor='k', legend_kwds={'label': "Acquisition Datetime"})
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(c=cam_final_gdf['dt'].dt.date, cmap='inferno', legend='True', edgecolor='k', legend_kwds={'label': "Acquisition Datetime"})
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)

## Compare with TLE

In [ ]:
tle_fn = 'test_tle_ecef_xyz.csv'
tle_cols = ['tle_x', 'tle_y', 'tle_z']

In [ ]:
cam_df = pd.read_csv(tle_fn, index_col='img')
cam_df = cam_df[cam_df.index.to_series().str.contains("pregeoreferenced")==False]

In [ ]:
cam_tle_delta = cam_final_gdf[['x','y','z']] - cam_df.values
cam_final_gdf['tle_diff_m'] = np.sqrt(np.square(cam_tle_delta).sum(axis=1))

In [ ]:
cam_gdf = gpd.GeoDataFrame(cam_df, geometry=gpd.points_from_xy(cam_df['ecef_x'], cam_df['ecef_y'], cam_df['ecef_z'], crs='EPSG:4978'))

In [ ]:
cam_gdf

In [ ]:
#Do a proper join
#cam_df.index.to_series().str.split('/', expand=True)[1].str.split('.', expand=True)[0].values

In [ ]:
cam_final_gdf['tle_diff_m']

In [ ]:
cam_tle_delta

In [ ]:
cam_final_gdf.head()

In [ ]:
plot_kw = {'markersize':10}
ax = cam_final_gdf.to_crs(map_crs).plot(color='b', label='Final', **plot_kw)
cam_gdf.to_crs(map_crs).plot(ax=ax, color='r', label='TLE', **plot_kw)
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ax.legend()
ctx.add_basemap(ax, **ctx_kwargs)
#ax.set_aspect('equal')

In [ ]:
ax = cam_final_gdf.plot.scatter(x='mx', y='my', c='dt_diff_days', s=36, cmap='inferno', edgecolor='k')
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)
ax.set_title('Time offset relative to '+min_dt_str)

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(vmin=0, vmax=20, column='global_id', cmap='tab20', legend='True', edgecolor='k', legend_kwds={'label': "BlackSky Satellite ID"})
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)

In [ ]:
plot_kw = {'markersize':10}
ax = cam_init_gdf.to_crs(map_crs).plot(color='r', label='Initial', **plot_kw)
cam_final_gdf.to_crs(map_crs).plot(ax=ax, color='b', label='Final', **plot_kw)
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ax.legend()
ctx.add_basemap(ax, **ctx_kwargs)
#ax.set_aspect('equal')

In [ ]:
#ax = cam_final_gdf.to_crs(map_crs).plot(column='diff_m', norm=matplotlib.colors.LogNorm(), legend='True', legend_kwds={'label': "Position Difference (m)"})
#ax = cam_final_gdf.to_crs(map_crs).plot(column='diff_m_2', norm=matplotlib.colors.LogNorm(), legend='True', legend_kwds={'label': "Position Difference (m)"})
#centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
#ctx.add_basemap(ax, **ctx_kwargs)

### Compute rotation delta

In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
R_init = R.from_matrix(cam_init_gdf[['r11','r12','r13','r21','r22','r23','r31','r32','r33']].values.reshape((cam_init_gdf.shape[0],3,3)))
R_final = R.from_matrix(cam_final_gdf[['r11','r12','r13','r21','r22','r23','r31','r32','r33']].values.reshape((cam_final_gdf.shape[0],3,3)))

In [ ]:
#R_init.as_euler('ZYX', degrees=True)
#R_final.as_euler('ZYX', degrees=True)

In [ ]:
eul_diff = (R_init.as_euler('ZYX', degrees=True) - R_final.as_euler('ZYX', degrees=True))

In [ ]:
cam_final_gdf['diff_deg'] = np.sqrt(np.square(eul_diff).sum(axis=1))

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(column='diff_deg', legend='True', legend_kwds={'label': "Orientation Difference (deg)"})
centroid_gdf.plot(ax=ax, marker='*', color='w', edgecolor='k')
ctx.add_basemap(ax, **ctx_kwargs)

In [ ]:
def cam_diff_plot(log=False):
    f, axa = plt.subplots(1,2, figsize=(10,3), sharex=True, sharey=True)
    norm=None
    if log:
        norm=matplotlib.colors.LogNorm()
    #plot_kw = {'norm':norm, 's':1, 'legend':True, 'legend_kwds':{'label': col}}
    cam_final_gdf.to_crs(map_crs).plot(ax=axa[0], norm=norm, column='diff_m', legend='True', legend_kwds={'label': "Position Difference (m)"})
    centroid_gdf.plot(ax=axa[0], marker='*', color='w', edgecolor='k')
    ctx.add_basemap(ax=axa[0], **ctx_kwargs)
    if log:
        norm=matplotlib.colors.LogNorm()
    cam_final_gdf.to_crs(map_crs).plot(ax=axa[1], norm=norm, column='diff_deg', legend='True', legend_kwds={'label': "Orientation Difference (deg)"})
    centroid_gdf.plot(ax=axa[1], marker='*', color='w', edgecolor='k')
    ctx.add_basemap(ax=axa[1], **ctx_kwargs)
    axa[0].set_title(f'Position Difference (m)')
    axa[1].set_title(f'Orientation Difference (deg)')
    plt.tight_layout()

In [ ]:
cam_diff_plot()

In [ ]:
cam_diff_plot(log=True)

## Geoplot tests for KDE

In [ ]:
#import geoplot as gplt
#import geoplot.crs as gcrs

In [ ]:
#ax = gplt.pointplot(mapproj_match_offset, projection=gcrs.AlbersEqualArea(), s=1)
#gplt.kdeplot(mapproj_match_offset[['mapproj_ip_dist_meters','geometry']], projection=gcrs.AlbersEqualArea(), ax=ax)

## Convergence angles

In [ ]:
conv_txt = ba_prefix+'-convergence_angles.txt'

In [ ]:
conv_cols = ['img1','img2','conv_25','conv_50','conv_75','num_angles']
conv = pd.read_csv(conv_txt, delimiter=' ', skiprows=1, header=0, names=conv_cols, index_col=False)
conv_valid = conv[conv['num_angles'] != 0]

In [ ]:
conv_valid.reset_index().plot.scatter(x='index', y='conv_50', c='num_angles', cmap='inferno')

In [ ]:
f, ax = plt.subplots()
m = ax.scatter(conv_valid.index, conv_valid['conv_50'], c=conv_valid['num_angles'], norm=matplotlib.colors.LogNorm())
plt.colorbar(m)

#### Testing Rotation Distance
http://www.boris-belousov.net/2016/12/01/quat-dist/ 

In [ ]:
R_final.as_matrix()[1]

In [ ]:
R_final.as_matrix()[1].T

In [ ]:
np.transpose(R_final.as_matrix(), axes=(0,2,1))[1]

In [ ]:
R_diff = R_init.as_matrix() * np.transpose(R_final.as_matrix(), axes=(0,2,1))

In [ ]:
R_diff[1]

In [ ]:
np.trace(R_diff, axis1=1, axis2=2)

In [ ]:
np.radians((np.trace(R_diff, axis1=1, axis2=2) - 1)/2)

In [ ]:
np.degrees(np.arccos(np.radians((np.trace(R_diff, axis1=1, axis2=2) - 1)/2)))